# 0. Install Dependencies





In [1]:
#!pip install rl-agents==0.1.1
#!pip install tensorflow
#!pip install gym
#!pip install keras
#!pip install --user keras-rl2
#!pip install pygame
#conda install swig -> this should be installed in the Anaconda command line
#conda install -c conda-forge box2d-py ->  this should be installed in the Anaconda command line

# 1. Test Random Environment with OpenAI Gym




In [2]:
#import tensorflow as tf
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import gym
import random

In [3]:
#Game being used
env = gym.make('LunarLander-v2')# ->https://gymnasium.farama.org/environments/box2d/lunar_lander/


states = env.observation_space.shape[0]

#Number of actions available.
actions = env.action_space

In [4]:
print(states)
print(actions.n)


8
4


In [5]:
#Env setting with random behaviour
episodes = 10
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0
        
    while not done:
        env.render()
        n_state, reward, done, info = env.step(env.action_space.sample())
        score+=reward
    print('Episode:{} Score:{}'.format(episode,score))
    
env.close()

Episode:1 Score:-114.8986943646137
Episode:2 Score:-75.1088244748039
Episode:3 Score:-127.97116196571781
Episode:4 Score:-64.01866857137395
Episode:5 Score:-319.84008999313244
Episode:6 Score:-82.25442151952902
Episode:7 Score:-93.66057245408658
Episode:8 Score:-186.33578532754603
Episode:9 Score:-85.2483761817838
Episode:10 Score:-286.88224362378065


# 2. Create a Deep Learning Model with Keras



In [6]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam

from keras import __version__
tf.keras.__version__ = __version__

from rl.agents import SARSAAgent #check diferent agents -> https://keras-rl.readthedocs.io/en/latest/
from rl.agents import DQNAgent #check diferent agents -> https://keras-rl.readthedocs.io/en/latest/
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


In [7]:
def build_model(states, actions):
    model = Sequential() 
    model.add(Flatten(input_shape=(1,) + env.observation_space.shape)) 
    model.add(Dense(200,activation='relu'))
    model.add(Dense(200,activation='relu'))
    model.add(Dense(actions,activation='linear'))    
    return model

In [8]:
#del model #-> Uncoment if Sequential error appears after building the agent

In [9]:
model = build_model(states,actions.n)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 128)               1152      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 132       
Total params: 11,620
Trainable params: 11,620
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras Neural Network




In [11]:
def build_agent(model, actions):
    policy = EpsGreedyQPolicy()
    #earlystop = EarlyStopping(monitor = 'episode_reward', min_delta=.1, patience=5, verbose=1, mode='auto') 
    memory = SequentialMemory(limit=50000,window_length=1)
    #callbacks = [earlystop] 
    nb_steps_warmup = 1000 
    target_model_update = .02 
    #gamma = .99 
    #epochs = training_steps/1000 
    #decay = float(lr/epochs) 
    dqn = DQNAgent(model=model, nb_actions=actions, memory=memory, nb_steps_warmup=nb_steps_warmup, target_model_update = target_model_update, policy=policy)
    return dqn

In [12]:
 #Adam._name = 'hey' ## use in case of error mentioning this parameter as null-
dqn = build_agent(model,actions.n)

lr = .0001 
dqn.compile(Adam(learning_rate=lr), metrics=['mae'],)

training_steps = 2000000

dqn.fit(env, nb_steps=training_steps, visualize=False, verbose=1) 

Training for 2000000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 271s 27ms/step - reward: -0.9349
43 episodes - episode_reward: -219.430 [-539.268, -2.123] - loss: 13.084 - mae: 32.767 - mean_q: -5.392

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 317s 32ms/step - reward: -0.0534
11 episodes - episode_reward: -37.325 [-175.087, 186.687] - loss: 3.925 - mae: 31.711 - mean_q: 30.179

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 310s 31ms/step - reward: -0.0055
11 episodes - episode_reward: -11.473 [-234.867, 175.665] - loss: 3.262 - mae: 29.404 - mean_q: 32.764

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 314s 31ms/step - reward: 0.03440s - reward: 0.034
10 episodes - episode_reward: 35.986 [-76.513, 240.205] - loss: 3.525 - mae: 28.646 - mean_q: 33.802

Interval 5 (40000 steps performed)
10000/10000 [==============================] -

10000/10000 [==============================] - 153s 15ms/step - reward: 0.5559
22 episodes - episode_reward: 246.709 [115.663, 294.787] - loss: 5.067 - mae: 45.695 - mean_q: 61.002

Interval 37 (360000 steps performed)
10000/10000 [==============================] - 156s 16ms/step - reward: 0.5680
25 episodes - episode_reward: 230.164 [-230.948, 301.523] - loss: 5.308 - mae: 46.812 - mean_q: 62.433

Interval 38 (370000 steps performed)
10000/10000 [==============================] - 157s 16ms/step - reward: 0.48642s - reward: 0.482 - ETA: 
24 episodes - episode_reward: 200.434 [-130.461, 294.533] - loss: 5.025 - mae: 46.105 - mean_q: 61.479

Interval 39 (380000 steps performed)
10000/10000 [==============================] - 161s 16ms/step - reward: 0.50143s - reward: - ETA: 2s - rewa - ETA: 1s -  - ETA: 1s - reward:  - ETA: 0s -
22 episodes - episode_reward: 229.053 [-5.638, 299.016] - loss: 7.422 - mae: 48.030 - mean_q: 63.757

Interval 40 (390000 steps performed)
10000/10000 [=========

10000/10000 [==============================] - 5667s 567ms/step - reward: 0.5544
35 episodes - episode_reward: 158.420 [-43.493, 293.137] - loss: 7.413 - mae: 52.239 - mean_q: 69.539

Interval 72 (710000 steps performed)
10000/10000 [==============================] - 235s 23ms/step - reward: 0.3605
26 episodes - episode_reward: 137.791 [-531.280, 301.426] - loss: 7.255 - mae: 53.500 - mean_q: 71.199

Interval 73 (720000 steps performed)
10000/10000 [==============================] - 171s 17ms/step - reward: 0.3623
29 episodes - episode_reward: 126.777 [-195.320, 307.376] - loss: 8.199 - mae: 51.410 - mean_q: 68.585

Interval 74 (730000 steps performed)
10000/10000 [==============================] - 176s 18ms/step - reward: -0.8796
42 episodes - episode_reward: -213.498 [-1237.516, 290.463] - loss: 14.063 - mae: 54.695 - mean_q: 73.056

Interval 75 (740000 steps performed)
10000/10000 [==============================] - 174s 17ms/step - reward: 0.2568
33 episodes - episode_reward: 77.609

In [13]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 278.615, steps: 240
Episode 2: reward: 228.422, steps: 212
Episode 3: reward: 287.743, steps: 311
Episode 4: reward: 245.099, steps: 224
Episode 5: reward: 285.469, steps: 204
Episode 6: reward: 278.986, steps: 286
Episode 7: reward: 251.033, steps: 370
Episode 8: reward: 267.478, steps: 272
Episode 9: reward: 88.778, steps: 1000
Episode 10: reward: 274.999, steps: 270
Episode 11: reward: 265.813, steps: 244
Episode 12: reward: 245.879, steps: 263
Episode 13: reward: 264.490, steps: 236
Episode 14: reward: 274.809, steps: 296
Episode 15: reward: 291.175, steps: 278
Episode 16: reward: 277.914, steps: 229
Episode 17: reward: 242.387, steps: 211
Episode 18: reward: 269.509, steps: 226
Episode 19: reward: 271.139, steps: 336
Episode 20: reward: 300.252, steps: 318
Episode 21: reward: 245.632, steps: 221
Episode 22: reward: 239.106, steps: 614
Episode 23: reward: 287.416, steps: 259
Episode 24: reward: 271.385, steps: 259
Episode 25: reward: 

In [14]:
#lets test again with more 15 episodes
_ = dqn.test(env, nb_episodes=15, visualize=True)
env.close()

Testing for 15 episodes ...
Episode 1: reward: 264.158, steps: 243
Episode 2: reward: 159.949, steps: 1000
Episode 3: reward: 246.209, steps: 230
Episode 4: reward: 246.213, steps: 191
Episode 5: reward: 272.052, steps: 376
Episode 6: reward: 230.638, steps: 295
Episode 7: reward: 275.117, steps: 299
Episode 8: reward: 283.678, steps: 246
Episode 9: reward: 275.258, steps: 242
Episode 10: reward: 291.510, steps: 328
Episode 11: reward: 289.308, steps: 268
Episode 12: reward: 265.099, steps: 309
Episode 13: reward: 274.676, steps: 306
Episode 14: reward: 247.326, steps: 323
Episode 15: reward: 260.874, steps: 249


In [15]:
dqn.save_weights('LunarLander-v2_weights.h5f',overwrite=True)